In [57]:
import numpy as np 
import pandas as pd
from scipy.stats import f
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.spatial.distance import mahalanobis
import os

### Название признаковых переменных

In [58]:
FEATURES = ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9"]

### Считывание данных

In [61]:
data = pd.read_excel('ЛР2.xlsx')
data.head()

,Регионы,X1,X2,X3,X4,X5,X6,X7,X8,X9,Обуч
0,Алтайский край,0.550909,-0.456354,-0.625030,-0.354471,1.077778,1.051521,0.610943,-0.294640,-0.816480,3.0
1,Амурская область,0.864949,0.767179,0.199023,0.241207,2.142743,1.169850,0.516397,0.143576,0.086356,3.0
2,Архангельская область,-0.262225,0.506562,0.289679,-0.606353,-0.355807,0.443996,0.043666,0.123418,1.236121,NaN
3,Астраханская область,0.188435,1.431635,-0.530478,0.028876,-0.695752,-0.045192,-0.271488,-0.136165,-0.392070,NaN
4,Белгородская область,-1.154079,-0.780811,0.010769,0.642145,-0.368769,-0.845349,0.327305,-0.150557,0.320792,NaN


### Переименуем колонки

In [63]:
columns = ["regions"] + FEATURES + ["Train"]
columns

['regions', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'Train']

In [64]:
df = data.set_axis(columns, axis=1, inplace=False)
df.head()

,regions,X1,X2,X3,X4,X5,X6,X7,X8,X9,Train
0,Алтайский край,0.550909,-0.456354,-0.625030,-0.354471,1.077778,1.051521,0.610943,-0.294640,-0.816480,3.0
1,Амурская область,0.864949,0.767179,0.199023,0.241207,2.142743,1.169850,0.516397,0.143576,0.086356,3.0
2,Архангельская область,-0.262225,0.506562,0.289679,-0.606353,-0.355807,0.443996,0.043666,0.123418,1.236121,NaN
3,Астраханская область,0.188435,1.431635,-0.530478,0.028876,-0.695752,-0.045192,-0.271488,-0.136165,-0.392070,NaN
4,Белгородская область,-1.154079,-0.780811,0.010769,0.642145,-0.368769,-0.845349,0.327305,-0.150557,0.320792,NaN


### Обучающая выборка

In [65]:
TRAIN_SAMPLES = {}

#### Сформируем хэш с обучающей выборкой, где ключ - номер класса, а значение - массив с номерами строк исходных данных

In [66]:
train_data = df.dropna()

In [67]:
for i in train_data['Train'].unique():
    TRAIN_SAMPLES[int(i)] = list(train_data[train_data['Train'] == i].index)
TRAIN_SAMPLES

{3: [0, 1, 14, 16],
 4: [7, 9, 15, 19, 23, 28, 37, 38, 40, 68, 69, 74, 79],
 1: [10, 11],
 2: [18, 25, 52, 60],
 6: [31, 33, 64],
 5: [47, 78]}

In [68]:
def get_train_data(data, features, train_samples=None):  #объявление функции принимает 3 параметра последние не обязательно вводить (3 это обучающая выборка) фичерс это Х загаловки а дата это таблица с данными
    train_data = pd.DataFrame() # создаётся переменная трейн дата( новый дата фрейм дата фрейм это таблица ) 
    for cls, samples in train_samples.items(): # заходим в цикл переменная cls будет последовательно принемать значения номеров классов то есть каждый цикл это следующий номер класса, sempls бедет принимать список номеров строк которые принадлежат этому классу   точка айтемс это пара значений (ключ и значение) для словаря train samples ключ это номер класса из обучающей выборки а значение это номера строк этого класса из обучающей выборки  
        train_samps = data[features].loc[samples] #train_samps это новая таблица в которую мы записываем значения(Х) которое входят в обучающаю выборку (значение номер класса и так до конца)
        train_samps["Class"] = cls # это доп колонка номер класса в которую вносится номер класса к которому принадлежит внесённый X
        train_data = pd.concat([train_data, train_samps]) # сохраняем полоученные строчки в train_data а train_samps в котором они были до этого очищаем
    train_data = train_data.astype({"Class": 'int32'}) #  в train_data присваиваем итоговую таблицу с обучающей выборкой и показываем что в колонке класс данные типа интедгер
    return train_data 

In [69]:
df = df[FEATURES]

train_data = get_train_data(df, FEATURES, TRAIN_SAMPLES)
print("Обучающая выборка")
df["Train sample"] = train_data.Class
train_data.tail()

Обучающая выборка


,X1,X2,X3,X4,X5,X6,X7,X8,X9,Class
31,2.051556,1.475578,2.703760,2.301340,0.788909,0.454324,-0.618158,0.648758,2.497777,6
33,1.527502,0.329709,0.967902,1.219572,-0.076838,-0.325896,-0.334519,0.583902,2.465808,6
64,2.354763,1.639920,1.988849,1.125183,1.333398,1.208625,-0.271488,-0.053325,1.811923,6
47,-1.187656,-0.743990,-0.467231,-1.414526,-1.381735,-1.720204,-2.603629,-0.622736,-0.986978,5
78,-1.757579,-1.683297,-0.573186,-1.426533,-1.534643,-1.826316,-2.603629,-0.588789,-0.895482,5


### Функции рассчета ковариционной матрицы

In [70]:
def scatter_matrix(samples):  
    if isinstance(samples, pd.Series):  
        samples = samples.to_frame()
    d = samples - samples.mean() 
    res = np.zeros((d.shape[1], d.shape[1])) 
    for _, row in d.iterrows(): 
        col = row.to_frame() 
        res += col @ col.T  
    return res

In [71]:
def classes_scatter_matrix(samples, labels): 
    A = np.zeros((samples.shape[1], samples.shape[1])) 
    for cls in labels.unique():
        A += scatter_matrix(samples[labels == cls])  
    return A

### Ковариционная матрица

In [72]:
cov = pd.DataFrame(
    classes_scatter_matrix(train_data[FEATURES], train_data.Class) / (train_data.shape[0] - train_data.Class.unique().size),
    index=FEATURES,
    columns=FEATURES
)
print("Ковариационная матрица")
cov

Ковариационная матрица


,X1,X2,X3,X4,X5,X6,X7,X8,X9
X1,0.139237,0.110312,-0.018467,0.010569,0.037643,0.051790,0.017888,-0.019735,-0.008466
X2,0.110312,0.285442,-0.010625,0.025824,0.080172,0.062186,-0.022444,-0.001422,-0.004413
X3,-0.018467,-0.010625,0.215093,0.059839,0.050950,0.012171,-0.009298,0.002930,0.030894
X4,0.010569,0.025824,0.059839,0.095225,0.025718,-0.006015,-0.007217,-0.004415,0.005490
X5,0.037643,0.080172,0.050950,0.025718,0.104385,0.073368,0.005235,-0.015210,-0.013498
X6,0.051790,0.062186,0.012171,-0.006015,0.073368,0.194070,-0.036391,-0.019413,-0.038413
X7,0.017888,-0.022444,-0.009298,-0.007217,0.005235,-0.036391,0.102047,-0.015547,-0.033168
X8,-0.019735,-0.001422,0.002930,-0.004415,-0.015210,-0.019413,-0.015547,0.062053,0.063241
X9,-0.008466,-0.004413,0.030894,0.005490,-0.013498,-0.038413,-0.033168,0.063241,0.113174


### Средние значения. Дискриминантный анализ

In [73]:
lda = LinearDiscriminantAnalysis().fit(train_data[FEATURES], train_data.Class)  
means = pd.DataFrame(lda.means_, index=lda.classes_, columns=FEATURES)       
print("Средние значения")
means

Средние значения


,X1,X2,X3,X4,X5,X6,X7,X8,X9
1,-0.590655,3.151645,2.164142,1.347212,-1.203168,-1.511003,-0.570885,-0.598642,-0.382883
2,-0.528066,-0.446835,0.065588,-0.030526,-0.595842,-0.833722,0.083060,-0.464958,-0.650206
3,0.795865,0.215592,-0.297038,-0.149525,1.487604,1.370797,0.240637,0.098742,-0.027049
4,-0.079271,-0.081356,-0.423924,0.176119,-0.174697,-0.131187,0.741853,-0.362468,-0.441563
5,-1.472617,-1.213643,-0.520209,-1.420530,-1.458189,-1.773260,-2.603629,-0.605762,-0.941230
6,1.977941,1.148402,1.886837,1.548698,0.681823,0.445684,-0.408055,0.393112,2.258502


### Расстояния Махланобиса (обучающая выборка)

In [74]:
def find_mahl_sqr_dist(centers, samples, covr):                                                # функция принемает в себя дважды средние значения и один раз матрицу ковариций 
    res = pd.DataFrame(index=samples.index, columns=centers.index)                             # создаём новый датафрейм вверху заголовки это номера классов и слева заголовки тоже номера классов 
    for i in centers.index:                                                                    # двойной цикл идёт по одной и той же таблице 
        for j in samples.index:
            res[i][j] = mahalanobis(centers.loc[i], samples.loc[j], np.linalg.inv(covr)) ** 2  # вычисляется растояние махаланобиса в квадрате.      np.linalg.inv(covr)-возвращает матрицу обратную матрице ковариации       centers.loc[i] и samples.loc[j] возвращают i и j строки таблицы means(ср знач) и это значение записывается в ячейку  ij
    return res

In [93]:
cen_dis = find_mahl_sqr_dist(means, means, cov)
cen_dis

,1,2,3,4,5,6
1,0.0,173.392363,428.670035,207.805606,422.571896,322.260813
2,173.392363,0.0,91.78693,22.520582,185.716631,173.063007
3,428.670035,91.78693,0.0,68.653206,334.075048,147.903065
4,207.805606,22.520582,68.653206,0.0,322.514993,132.728837
5,422.571896,185.716631,334.075048,322.514993,0.0,517.958852
6,322.260813,173.063007,147.903065,132.728837,517.958852,0.0


### Функция Фишера

In [76]:
def get_def_coef(lda, features):
    return pd.DataFrame(
        np.vstack([lda.intercept_, lda.coef_.T]),                                # Сложение массивов  intercept и coef( транспонированный ) и получаем датафрейм содержащий в себе данный полученные в результате сложения массивов
        index=["Const"] + features,                                              # по строкам признаки Х1...Х9, coef
        columns=lda.classes_                                                     # по столбцам по номерам классов из дискр анализа 
    )

In [92]:
df_coef = get_def_coef(lda, FEATURES)
df_coef

,1,2,3,4,5,6
Const,-113.450387,-7.651849,-21.193645,-6.172528,-121.081795,-58.967531
X1,-20.008710,0.167182,10.261379,-5.569177,17.738970,11.741512
X2,29.760459,-0.758281,-11.568214,1.464763,-11.316718,-2.207807
X3,17.751747,2.887104,-6.183340,-3.388808,7.948729,1.946167
X4,5.749029,-3.773358,-4.574460,3.550764,-28.576907,10.962365
X5,-42.128989,-1.494791,27.838088,-3.615033,17.371101,-2.953996
X6,3.407218,-5.756524,-1.657513,5.326268,-35.349181,8.099530
X7,6.231033,-5.504002,-5.200786,10.271043,-58.865771,4.855022
X8,-22.043866,4.112851,21.250232,-4.263331,24.634941,-17.070392
X9,2.795574,-11.046870,-7.289762,4.624787,-47.794764,34.407557


### Априорные вероятности

In [78]:
print("Pi: ", lda.priors_)

Pi:  [0.07142857 0.14285714 0.14285714 0.46428571 0.07142857 0.10714286]


### Распределение по классам по результатам ДА

In [79]:
def LDA_predict(lda, x):# принимает в себя результаты линейного дискр анализа и значения признаков всех объектов( исходная таблица только с значениями X1...X9)
    return pd.DataFrame(    # функция считает распределение по классам (классификация масива тестовых векторов Х  )
        lda.predict(x),
        columns=["Class"],
        index=x.index
    )

In [91]:
lda_predict = LDA_predict(lda, data[FEATURES])
data["Result Lda"] = lda_predict
lda_predict

,Class
0,3
1,3
2,4
3,2
4,4
...,...
78,5
79,4
80,6
81,3


### Расстояние Махланобиса

In [85]:
samp_dist = find_mahl_sqr_dist(means, data[FEATURES], cov)
samp_dist

,1,2,3,4,5,6
0,417.653104,70.855752,9.058892,58.593599,294.930361,191.196832
1,452.254487,118.781469,7.807152,90.219385,383.962899,157.672896
2,219.498824,113.696133,164.99767,70.94767,456.217932,120.313379
3,125.186282,25.14581,118.764927,29.884233,252.40782,159.861169
4,205.216884,45.381377,111.222727,20.01058,355.240614,133.01212
...,...,...,...,...,...,...
78,434.267347,184.174518,331.156007,318.916965,1.203562,519.536276
79,197.828168,11.182912,71.910852,8.896696,242.623691,144.411322
80,578.577439,370.901224,232.019596,340.782906,669.580593,82.17401
81,1547.082873,1238.422776,1086.883717,1321.157682,1266.157436,1290.049948


In [87]:
def LDA_predict_probab(lda, x): # принимает в себя результаты линейного дискр анализа и значения признаков всех объектов( исходная таблица только с значениями X1...X9)
    return pd.DataFrame(       # возвращает апостериорные вероятности классификации в соответствии с каждым классом в массиве тестовых векторов  
        lda.predict_proba(x),
        columns=lda.classes_,
        index=x.index
    )

### Апосториорные вероятности

In [90]:
lda_post_prob = LDA_predict_probab(lda, data[FEATURES])
lda_post_prob

,1,2,3,4,5,6
0,9.415952e-90,3.810504e-14,1.000000e+00,5.695851e-11,4.195361e-63,2.110135e-40
1,1.543325e-97,7.984234e-25,1.000000e+00,4.133276e-18,1.041807e-82,2.148465e-33
2,8.503807e-34,1.604744e-10,1.162568e-21,1.000000e+00,3.362886e-85,4.400990e-12
3,7.247073e-23,7.668405e-01,3.593952e-21,2.331595e-01,1.715366e-50,3.211426e-30
4,9.333583e-42,9.526155e-07,4.804522e-21,9.999990e-01,2.470638e-74,6.686594e-26
...,...,...,...,...,...,...
78,9.149429e-95,3.710153e-40,4.495386e-72,6.642393e-69,1.000000e+00,4.183816e-113
79,1.319766e-42,8.933653e-02,5.809309e-15,9.106635e-01,2.473276e-52,7.868806e-31
80,1.074666e-108,2.683000e-63,3.858186e-33,3.024224e-56,1.862901e-128,1.000000e+00
81,5.861286e-101,1.240827e-33,1.000000e+00,4.364537e-51,5.890830e-40,5.728944e-45


### Пошаговый ДА с включением

In [94]:
def wilks_lambda(samples, labels):  # samples  - на каждой итерации мы передаём в функцию wilks lmbd dataframe сначала с одной колонкой Х1 постепенно увеличивая число колонок пока не дойдём до конца   labels - колонка с номерами классов 
    if isinstance(samples, pd.Series):  
        samples = samples.to_frame()
    # определитель матрицы рассеивания
    dT = np.linalg.det(scatter_matrix(samples))
    # определитель классовой матрицы рассеивания
    dE = np.linalg.det(classes_scatter_matrix(samples, labels))
    return dE / dT

In [95]:
def f_p_value(lmbd, n_obj, n_sign, n_cls): #  sign это число признаков вошедших в модель lmbd - значение лямбды( число)   n_obj - число объектов в обучающей выборке  n_cls - число классов в обучающей выборке 
    num = (1-lmbd)*(n_obj - n_cls - n_sign)
    den = lmbd * (n_cls - 1)
    f_value = num / den
    p = f.sf(f_value, n_cls-1, n_obj-n_cls-n_sign)
    return f_value, p

In [96]:
def forward(samples, labels, f_in = 1e-4):                                            # samples - значение признаков в обучаюзей выборке  labels - колонка с номерами классов  f_in точность( это F to inter в статистике)
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to enter", "P value"]             # создаётся список названий колонок таблицы 
    n_cls = labels.unique().size                                                      #число уникальных классов нашей обучающей выборки 
    n_obj = samples.shape[0]                                                          # число  объектов в обучающей выборке
                                                                                      # хранение пременных вне и в модели(е)
    out = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}   # создаётся словарик ключу(ключ показывает какой у нас шаг метода) 0 ставится в соответствие дата фрейм(пустой) с колонками из переменной st_colums и индексами(строки таблицы) Х1,,,Х9
    into = {0: pd.DataFrame(columns=st_columns, dtype=float)}                         # создаётся словарик ключу(ключ показывает какой у нас шаг метода) 0 ставится в соответствие дата фрейм(пустой) с колонками из переменной st_colums 
    step = 0                                                                          # шаг нашего метода 

    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # расчёт характеристик элементов вне модели
        for el in out[step].index: # el переменная счётчик по списку индексов датафрейма внутри out (Х1 Х2,,, Х9)
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)   # мы с датафрейма samples  берём значение из колонок в квадратных скобках список колонок который есть в inta для текущего шага + текущая колонку из счётчика el и эти значения помещаются в функцию вилкс лямбда 
            partial_lmbd = lmbda / model_lmbd   # 
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # расчёт характеристик элементов в моделе
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size-1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if out[step].index.size == 0 or out[step]["F to enter"].max() < f_in:
            break
        # добавление нового элемента
        el_to_enter = out[step]["F to enter"].idxmax()
        into[step+1] = into[step].append(out[step].loc[el_to_enter])
        out[step+1] = out[step].drop(index=el_to_enter)

        step += 1
    return into, out

In [97]:
into, out = forward(train_data[FEATURES], train_data.Class)
print("Forward stepwise")
for i, tab in into.items():
    print("Step: ", i)
    print(tab, end="\n\n")

C:\Users\User\AppData\Local\Temp\ipykernel_2600\3314717884.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step+1] = into[step].append(out[step].loc[el_to_enter])
C:\Users\User\AppData\Local\Temp\ipykernel_2600\3314717884.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step+1] = into[step].append(out[step].loc[el_to_enter])
C:\Users\User\AppData\Local\Temp\ipykernel_2600\3314717884.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step+1] = into[step].append(out[step].loc[el_to_enter])
C:\Users\User\AppData\Local\Temp\ipykernel_2600\3314717884.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step+1] 

Forward stepwise
Step:  0
Empty DataFrame
Columns: [Wilk's lmbd, Partial lmbd, F to enter, P value]
Index: []

Step:  1
    Wilk's lmbd  Partial lmbd  F to enter       P value
X7          1.0      0.093686   42.565182  1.374993e-10

Step:  2
    Wilk's lmbd  Partial lmbd  F to enter       P value
X7     0.106317      0.085458   44.947225  1.627944e-10
X9     0.093686      0.096979   39.108453  6.038003e-10

Step:  3
    Wilk's lmbd  Partial lmbd  F to enter       P value
X7     0.014113      0.106542   33.543909  4.588602e-09
X9     0.011492      0.130840   26.571621  3.449597e-08
X5     0.009086      0.165493   20.170140  3.423416e-07

Step:  4
    Wilk's lmbd  Partial lmbd  F to enter       P value
X7     0.002415      0.103596   32.880826  1.011248e-08
X9     0.001685      0.148452   21.797496  2.879569e-07
X5     0.001935      0.129289   25.591478  7.980101e-08
X2     0.001504      0.166383   19.038909  8.269415e-07

Step:  5
    Wilk's lmbd  Partial lmbd  F to enter       P value


In [109]:
forw_stepwise = into[len(into) - 3].index.tolist()                    # смотрим каждый шаг  выбираем последний шаг где p-value не превышает 0,05. смотрим сколько признаков на этом шаге вошло в модель
print(forw_stepwise, '\n')
forw_stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[forw_stepwise], train_data.Class)
forw_stepwise_coef = get_def_coef(forw_stepwise_lda, forw_stepwise)

['X7', 'X9', 'X5', 'X2', 'X3', 'X1', 'X6'] 



In [110]:
print("Функции Фишера")
forw_stepwise_coef

Функции Фишера


,1,2,3,4,5,6
Const,-102.836106,-8.320240,-20.479876,-4.261331,-96.170775,-47.362582
X7,0.981683,-3.974595,-0.337475,8.759126,-48.488108,-0.535835
X9,-12.222463,-7.903188,7.065175,1.411712,-28.208923,21.954187
X5,-37.512784,-2.921575,23.590683,-2.211563,7.573804,1.983948
X2,27.158480,-0.322545,-9.042096,1.006443,-8.816795,-4.102856
X3,20.560873,1.648208,-8.635246,-2.200035,-1.053419,5.844567
X1,-13.757680,-1.392316,4.376276,-4.005128,7.527984,17.530075
X6,-1.278517,-4.248324,2.632396,3.847545,-24.912804,2.942757


In [111]:
print("Pi: ", forw_stepwise_lda.priors_)

Pi:  [0.07142857 0.14285714 0.14285714 0.46428571 0.07142857 0.10714286]


In [112]:
forw_stepwise_pred = LDA_predict(forw_stepwise_lda, data[forw_stepwise])
print("Распределение")
print(forw_stepwise_pred.head())
data["Result forward"] = forw_stepwise_pred

Распределение
   Class
0      3
1      3
2      4
3      2
4      4


### Пошаговый ДА с исключением

In [115]:
def backward(samples, labels, f_r = 10.00):
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to remove", "P value"]
    n_cls = labels.unique().size
    n_obj = samples.shape[0]
    # хранение пременных вне и в модели(е)
    into = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}
    out = {0: pd.DataFrame(columns=st_columns, dtype=float)}
    step = 0

    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # расчёт характеристик элементов вне модели
        for el in out[step].index:
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)
            partial_lmbd = lmbda / model_lmbd
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # расчёт характеристик элементов в моделе
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size-1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if into[step].index.size == 0 or into[step]["F to remove"].min() > f_r:
            break
        # удаление элемента
        el_to_remove = into[step]["F to remove"].idxmin()
        out[step+1] = out[step].append(into[step].loc[el_to_remove])
        into[step+1] = into[step].drop(index=el_to_remove)

        step += 1
    return into, out

In [116]:
into, out = backward(train_data[FEATURES], train_data.Class)
print("Backward stepwise")
for i, tab in into.items():
    print("Step: ", i)
    print(tab, end="\n\n")

C:\Users\User\AppData\Local\Temp\ipykernel_2600\1390836427.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step+1] = out[step].append(into[step].loc[el_to_remove])
C:\Users\User\AppData\Local\Temp\ipykernel_2600\1390836427.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step+1] = out[step].append(into[step].loc[el_to_remove])
C:\Users\User\AppData\Local\Temp\ipykernel_2600\1390836427.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step+1] = out[step].append(into[step].loc[el_to_remove])
C:\Users\User\AppData\Local\Temp\ipykernel_2600\1390836427.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step+1] =

Backward stepwise
Step:  0
    Wilk's lmbd  Partial lmbd  F to remove   P value
X1     0.000035      0.586815     1.971518  0.145722
X2     0.000076      0.270526     7.550196  0.001260
X3     0.000034      0.605640     1.823205  0.172771
X4     0.000028      0.733212     1.018812  0.443265
X5     0.000055      0.377518     4.616862  0.010679
X6     0.000036      0.568055     2.129097  0.121907
X7     0.000142      0.144934    16.519176  0.000020
X8     0.000029      0.706196     1.164904  0.373870
X9     0.000043      0.475126     3.093179  0.043592

Step:  1
    Wilk's lmbd  Partial lmbd  F to remove       P value
X1     0.000047      0.600303     1.997476  1.374413e-01
X2     0.000130      0.215071    10.948863  1.400681e-04
X3     0.000052      0.534681     2.610819  6.849376e-02
X5     0.000074      0.379643     4.902159  7.397314e-03
X6     0.000046      0.605992     1.950561  1.451989e-01
X7     0.000380      0.073840    37.628545  5.717811e-08
X8     0.000042      0.664287     

In [117]:
back_stepwise = into[len(into) - 5].index.tolist()
back_stepwise

['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']

In [118]:
back_stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[back_stepwise], train_data.Class)
back_stepwise_coef = get_def_coef(back_stepwise_lda, back_stepwise)
print("Функции Фишера")
back_stepwise_coef

Функции Фишера


,1,2,3,4,5,6
Const,-111.322511,-7.795681,-21.240242,-5.130441,-97.457572,-52.917430
X1,-19.088998,-0.436468,9.529570,-5.001136,13.167324,13.495238
X2,29.824959,-0.800616,-11.619536,1.504600,-11.637332,-2.084817
X3,19.292964,1.875530,-7.409675,-2.436909,0.287745,4.884991
X5,-40.873064,-2.319113,26.838757,-2.839337,11.128228,-0.559172
X6,2.006764,-4.837340,-0.543182,4.461308,-28.387895,5.429116
X7,4.997945,-4.694668,-4.219626,9.509453,-52.736414,2.503744
X8,-21.707172,3.891863,20.982327,-4.055380,22.961324,-16.428377
X9,1.849967,-10.426223,-6.537349,4.040753,-43.094401,32.604454


In [119]:
print("Pi: ", back_stepwise_lda.priors_)

Pi:  [0.07142857 0.14285714 0.14285714 0.46428571 0.07142857 0.10714286]


In [121]:
back_stepwise_pred = LDA_predict(back_stepwise_lda, data[back_stepwise])
print("Распределение")
data["Result backward"] = back_stepwise_pred
back_stepwise_pred.head()

Распределение


,Class
0,3
1,3
2,4
3,2
4,4


In [122]:
data.to_excel(r'Итоговый результат.xlsx')